In [27]:
import openmeteo_requests
import pandas as pd
import requests_cache
from datetime import datetime, timedelta
from retry_requests import retry

In [28]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
yesterday = datetime.now() - timedelta(1)

In [29]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2020-01-01",
	"end_date": datetime.strftime(yesterday, '%Y-%m-%d'),
	"hourly": "temperature_2m",
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean"],
	"temperature_unit": "fahrenheit",
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.5483283996582°E 13.407821655273438°N
Elevation 38.0 m asl
Timezone b'America/New_York' b'EST'
Timezone difference to GMT+0 -18000 s
                     date  temperature_2m
0     2020-01-01 05:00:00       31.475300
1     2020-01-01 06:00:00       30.935301
2     2020-01-01 07:00:00       30.125299
3     2020-01-01 08:00:00       31.205299
4     2020-01-01 09:00:00       34.085300
...                   ...             ...
35947 2024-02-07 00:00:00             NaN
35948 2024-02-07 01:00:00             NaN
35949 2024-02-07 02:00:00             NaN
35950 2024-02-07 03:00:00             NaN
35951 2024-02-07 04:00:00             NaN

[35952 rows x 2 columns]


In [30]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

                    date  temperature_2m_max  temperature_2m_min  \
0    2020-01-01 05:00:00           39.305298           25.895300   
1    2020-01-02 05:00:00           41.555298           25.355301   
2    2020-01-03 05:00:00           43.445301           35.435299   
3    2020-01-04 05:00:00           41.555298           31.115299   
4    2020-01-05 05:00:00           38.765301           28.685301   
...                  ...                 ...                 ...   
1493 2024-02-02 05:00:00           48.035301           38.945301   
1494 2024-02-03 05:00:00           50.735302           41.825302   
1495 2024-02-04 05:00:00           49.475300           41.555298   
1496 2024-02-05 05:00:00           48.935303           46.505302   
1497 2024-02-06 05:00:00           50.645302           46.775299   

      temperature_2m_mean  
0               31.936552  
1               33.121548  
2               41.026546  
3               37.261551  
4               35.506554  
...            

In [32]:
daily_dataframe.to_csv("../data/weather_data.csv")